# Preparation of GWR - Data

## Libraries and Settings

In [2]:
# Libraries
import os
import json
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import date

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/Users/ricostaedeli/Documents/ZHAW/GIS/gis_unterricht/Gis_projekt


## Download GWR data

Source: https://public.madd.bfs.admin.ch

In [3]:
canton = "zh"
# Download latest file for the Canton of Zurich
url = f"https://public.madd.bfs.admin.ch/buildings_{canton}.geojson"
response = requests.get(url)

# Ensure response is valid
if response.status_code == 200:
    # Open file in write mode and write the response content
    with open('DataBig/buildings_ai.geojson', 'wb') as file:
        file.write(response.content)
else:
    print(f"Failed to download file, status code: {response.status_code}")

In [4]:
import geopandas as gpd

# Path to your GeoJSON file
geojson_path = f'DataBig/buildings_{canton}.geojson'
# Desired output CSV file path
csv_path = f'DataBig/output_{canton}.csv'

# Read the GeoJSON file
gdf = gpd.read_file(geojson_path)

gdf

,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,geometry
0,1,1004,1040,1271,2,Affoltern am Albis,ZH,POINT (2676523.000 1235843.000)
1,10,1004,1020,1110,2,Affoltern am Albis,ZH,POINT (2676440.038 1235973.538)
2,100,1004,1020,1110,2,Affoltern am Albis,ZH,POINT (2676765.706 1236101.658)
3,1000,1007,1030,1110,2,Affoltern am Albis,ZH,POINT (2676644.881 1236812.863)
4,10000,1004,1030,1121,291,Andelfingen,ZH,POINT (2693391.650 1272255.267)
...,...,...,...,...,...,...,...,...
393664,99995,1004,1040,0,198,Uster,ZH,POINT (2696792.976 1245085.637)
393665,99996,1004,1040,0,198,Uster,ZH,POINT (2696718.474 1245039.536)
393666,99997,1004,1040,0,198,Uster,ZH,POINT (2696722.040 1245043.472)
393667,99998,1004,1040,0,198,Uster,ZH,POINT (2696727.666 1245050.586)


In [5]:
gdf['geometry_wkt'] = gdf['geometry'].apply(lambda x: x.wkt)

# Assuming your data is in a CRS like EPSG:2056 (Swiss coordinate system)
gdf.crs = "EPSG:2056"  # Set the correct CRS

# Convert to geographic coordinates (WGS84)
gdf = gdf.to_crs(epsg=4326)

# Extract longitude and latitude into separate columns
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y


# Save the DataFrame to CSV
gdf.to_csv(csv_path, index=False)

gdf

,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,geometry,geometry_wkt,longitude,latitude
0,1,1004,1040,1271,2,Affoltern am Albis,ZH,POINT (8.44986 47.26906),POINT (2676523 1235843),8.449859,47.269056
1,10,1004,1020,1110,2,Affoltern am Albis,ZH,POINT (8.44879 47.27024),POINT (2676440.038 1235973.538),8.448785,47.270239
2,100,1004,1020,1110,2,Affoltern am Albis,ZH,POINT (8.45311 47.27135),POINT (2676765.706 1236101.658),8.453110,47.271354
3,1000,1007,1030,1110,2,Affoltern am Albis,ZH,POINT (8.45164 47.27776),POINT (2676644.881 1236812.863),8.451635,47.277764
4,10000,1004,1030,1121,291,Andelfingen,ZH,POINT (8.68035 47.59434),POINT (2693391.65 1272255.267),8.680355,47.594339
...,...,...,...,...,...,...,...,...,...,...,...
393664,99995,1004,1040,0,198,Uster,ZH,POINT (8.71967 47.34952),POINT (2696792.976 1245085.637),8.719673,47.349516
393665,99996,1004,1040,0,198,Uster,ZH,POINT (8.71868 47.34911),POINT (2696718.474 1245039.536),8.718677,47.349113
393666,99997,1004,1040,0,198,Uster,ZH,POINT (8.71873 47.34915),POINT (2696722.04 1245043.472),8.718725,47.349147
393667,99998,1004,1040,0,198,Uster,ZH,POINT (8.71880 47.34921),POINT (2696727.666 1245050.586),8.718801,47.349211


In [6]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 23.4.0
Datetime: 2024-05-11 17:20:02
Python Version: 3.12.3
-----------------------------------
